In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/MBC 뉴스/210825

/content/drive/Shareddrives/MBC 뉴스/210825


In [ ]:
!pwd

/content/drive/Shareddrives/MBC 뉴스/210825


In [ ]:
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import datetime
import time
from tqdm import notebook
import re
import glob
import pandas as pd

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2932736/45929032 bytes (6.4%)6455296/45929032 bytes (14.1%)10059776/45929032 bytes (21.9%)12976128/45929032 bytes (28.3%)16375808/45929032 bytes (35.7%)19931136/45929032 bytes (43.4%)23601152/45929032 bytes (51.4%)27246592/45929032 bytes (59.3%)30859264/45929032 bytes (67.2%)34119680/45929032 bytes (74.3%)37675008/45929032 bytes (82.0%)41181184/45929032 bytes (89.7%)

In [ ]:
!pip install pytube

     |████████████████████████████████| 55 kB 2.6 MB/s 


In [ ]:
from pytube import YouTube
from pytube import Playlist

In [ ]:
# Create_Folder 함수 : 폴더 생성 함수
def Create_Folder_num(Clean_File_name):
    print(Clean_File_name, '처리하는 중')

    # YouTube 객체에서 날짜 정보 불러오기
    # 200번 이하의 파일이름은 Pitchfork 
    if int(Clean_File_name) <= 200:
      pl = Playlist("https://www.youtube.com/playlist?list=PLrIyFAtNxiQZRuVlAPQbhTpyiO00vk8ww")
      yt = YouTube(pl.video_urls[int(Clean_File_name)])
    
    # 1000번 이상의 파일이름은 Vogue
    elif int(Clean_File_name) >= 1000:
      pl = Playlist("https://www.youtube.com/playlist?list=PLztAHXmlMZFS9ZN7GTlZ2UOB2JmxICdt8")
      yt = YouTube(pl.video_urls[int(Clean_File_name)-1000])


    # 기준 시간 : 1년 1월 1일
    standard_time = datetime.datetime(1,1,1)

    # 결과 시간 = 파일 생성 시간 - 기준 시간
    result_time = int((yt.publish_date - standard_time).total_seconds())
    result_time = str(result_time)
    # print(result_time)

    # 000000.mp4 ~ 000200.mp4 : Pitchfork 영상 폴더
    # if int(Clean_File_name) <= 200:
    #   create_folder_dir = './main/Pitchfork/' + str(result_time)

    # 001000.mp4 ~ : Vogue 영상 폴더
    # elif int(Clean_File_name) >= 1000:
    #   create_folder_dir = './main/Vogue/' + str(result_time)

    # try: 
    #     if not os.path.exists(create_folder_dir):
    #         os.makedirs(create_folder_dir)
    #         print(create_folder_dir, '폴더 생성')
            
    # except OSError:
    #     print(create_folder_dir, ': 폴더 생성 오류')

    return result_time
  

In [ ]:
import moviepy.editor as mp
import re
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os

In [ ]:
import ast
from moviepy.video.io.VideoFileClip import VideoFileClip

In [ ]:
def get_timeline(script): 
  f = open(script, mode='rt', encoding='utf-8')
  sub = f.read()
    
  # 문자열 상태의 대본 -> type 자동 변환
  sub = ast.literal_eval(sub)

  timeline= []
  
  for i in range(0, len(sub)-1):

    # '니다' 직전까지만 끊기 #문제는 문장을 어디까지로 봐야할 지 모르겠다는 것임...
    if "you" in sub[i]['text'] or "they" in sub[i]['text']:
      start = sub[i]['start']
      end = sub[i+1]['start']

      timeline.append((start, end))
      break
    
    elif " " in sub[i]['text'] and "music" not in sub[i]['text'] and "Music" not in sub[i]['text'] and "drum" not in sub[i]['text']:
      start = sub[i]['start']
      end = sub[i+1]['start']

      timeline.append((start, end))


    # elif "니다" in sub2[i-1]['text'] :
    #   break

      
      # timeline.append((start, end))
    #리스트에 튜플형식으로 start, end 만드는 게 좋을 것 같다.
  print(timeline)
  
  return timeline


In [ ]:
#대본 만들기
def get_script(script, Clean_File_name):
  f = open(script, mode='rt', encoding='utf-8')
  sub = f.read()
  
  sub = ast.literal_eval(sub)
  text = []

  count = 0
  for i in range(0, len(sub)-1):
    if "you" in sub[i]['text'] or "they" in sub[i]['text']:
      txt = sub[i]['text']

      script_name = './Preprocess/TXT/' +Clean_File_name + '_' + str(count) + '.txt'
      f = open(script_name, 'w')
      f.write(txt)
      f.close()

      count += 1
      
      break

    elif " " in sub[i]['text'] and "music" not in sub[i]['text'] and "Music" not in sub[i]['text'] and "drum" not in sub[i]['text']:
      txt = sub[i]['text']

      script_name = './Preprocess/TXT/' +Clean_File_name + '_' + str(count) + '.txt'
      f = open(script_name, 'w')
      f.write(txt)
      f.close()
      
      count += 1

    # elif "니다" in sub2[i-1]['text'] :
    #   break

        
    
        # script_name = './Preprocess/TXT/' +Clean_File_name + '_' + str(i) + '.txt'
        # f = open(script_name, 'w')
        # f.write(txt)
        # f.close()

In [ ]:
# face_crop_Vogue 함수 : Vogue 영상에 맞도록 face crop 기능을 최적화하였음
def face_crop_Vogue(Clean_file_name):
    whole_pos = pd.DataFrame({'x':[], 'y':[], 'w':[], 'h':[]})
    file_name = './Preprocess/CUT_VID/' + Clean_file_name + '.mp4'

    # 얼굴 인식 모델 가중치 파일
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
    print(Clean_file_name, ': 처리하는 중')

    # [ 1단계 ] 첫 프레임에서 얼굴이 나타나는지 확인하기
    video1 = cv2.VideoCapture(file_name)
    
    ret,frame = video1.read()
    frame[0]

    # 얼굴 여러 개 인식시키기
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 
                                        scaleFactor=1.3, 
                                        minNeighbors=5, minSize=(20,20))
    
    video1.release()

    # 얼굴이 하나 이상 나오는 경우에만 얼굴 crop하기
    if len(faces) > 0: 
      video2 = cv2.VideoCapture(file_name)

      # [ 2단계 ] 얼굴 크롭 좌표 구하기
      while True:
        # 캡처 이미지 불러오기
        ret,img = video2.read()
          
        if ret == True:
          # 그레이 스케일 변환
          gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
          # cascade 얼굴 탐지 알고리즘 
          faces = face_cascade.detectMultiScale(gray,            
                                                scaleFactor= 1.3,
                                                minNeighbors=5,  minSize=(50,50))
          
          for face in faces:
            x, y, w, h = face
            # print(x, y, w, h)
            pos = pd.DataFrame({'x':[x], 'y':[y], 'w':[w], 'h':[h]})
            whole_pos = pd.concat([whole_pos, pos])

        # ret 값이 False일 경우에는 탈출
        else:
          break

      if whole_pos['w'].max() - whole_pos['w'].min() < 100:
        # 카메라 워킹 시를 대비하기 위한 초점 고정 코드
        x = int(whole_pos['x'].min())
        y = int(whole_pos['y'].min())
        w = int(whole_pos['w'].max())
        h = int(whole_pos['h'].max())

        print(x, y, w, h)

        video2.release()


        # [ 3단계 ] 얼굴 부분 크롭 후 avi 파일로 저장 (소리 X -> sound_synthesis 부분에서 합성)
        video3 =  cv2.VideoCapture(file_name)

        # 저장할 파일 양식 지정
        fps = video3.get(cv2.CAP_PROP_FPS)
        fourcc = cv2.VideoWriter_fourcc(*'DIVX')
        # width = video3.get(cv2.CAP_PROP_FRAME_WIDTH)
        # height = video3.get(cv2.CAP_PROP_FRAME_HEIGHT)
        size = (160,160) # 해상도 160*160 지정

        new_file_name = './Preprocess/FACE_VID/' + Clean_file_name + '.avi' 
        # new_file_name = re.sub(pattern='mp4', repl='avi', string=file_name)
        # save_file_name = './Preprocess/FACE_VID/%s.avi' %Clean_File_name
        out = cv2.VideoWriter(new_file_name, fourcc, fps, size)
        # out = cv2.VideoWriter(save_file_name, fourcc, fps, size)


        # 파일 처리 작업
        while (video3.isOpened()):
            ret, frame = video3.read()    

            if not ret:
                break

            cropped = frame[y :y + 2*h , 
                            x :x + 2*w ]

            cropped = cv2.resize(cropped, 
                                dsize=size, 
                                interpolation=cv2.INTER_AREA)      
            
            # 위에서 자른 얼굴 위치를 토대로 프레임 저장 (연속적 프레임이므로 영상 형태)
            out.write(cropped)

        # cropped 이미지 더 없으면 resize 함수 작동 중지 (혹시 모를 오류 방지)
        out.release()
        video3.release()

        return Clean_file_name
        
          
        

In [ ]:
# face_crop_Pitchfork 함수 : Pitchfork 영상에 맞도록 face crop 기능을 최적화하였음
# Pitchfork 영상 스타일 : 줌인/줌아웃 굉장히 빈번함
def face_crop_Pitchfork(Clean_file_name):
    whole_pos = pd.DataFrame({'x':[], 'y':[], 'w':[], 'h':[]})
    file_name = './Preprocess/CUT_VID/' + Clean_file_name + '.mp4'

    # 얼굴 인식 모델 가중치 파일
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
    print(file_name, ': 처리하는 중')

    video1 = cv2.VideoCapture(file_name)

    # [ 1단계 ] 얼굴 크롭 좌표 구하기
    while True:
      # 캡처 이미지 불러오기
      ret,img = video1.read()
        
      if ret == True:
        # 그레이 스케일 변환
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        # cascade 얼굴 탐지 알고리즘 
        faces = face_cascade.detectMultiScale(gray,            
                                              scaleFactor= 1.3,
                                              minNeighbors=5,  minSize=(50,50))
        
        for face in faces:
          x, y, w, h = face
          # print(x, y, w, h)
          pos = pd.DataFrame({'x':[x], 'y':[y], 'w':[w], 'h':[h]})
          whole_pos = pd.concat([whole_pos, pos])

      # ret 값이 False일 경우에는 탈출
      else:
        break

    if whole_pos['w'].max() - whole_pos['w'].min() < 30:
      # 카메라 워킹 시를 대비하기 위한 초점 고정 코드
      x = int(whole_pos['x'].min())
      y = int(whole_pos['y'].min())
      w = int(whole_pos['w'].max())
      h = int(whole_pos['h'].max())

      # print(x, y, w, h)

      video1.release()

    
      # [ 2단계 ] 얼굴 부분 크롭 후 avi 파일로 저장 (소리 X -> sound_synthesis 부분에서 합성)
      video2 =  cv2.VideoCapture(file_name)

      # 저장할 파일 양식 지정
      fps = video2.get(cv2.CAP_PROP_FPS)
      fourcc = cv2.VideoWriter_fourcc(*'DIVX')
      # width = video3.get(cv2.CAP_PROP_FRAME_WIDTH)
      # height = video3.get(cv2.CAP_PROP_FRAME_HEIGHT)
      size = (160,160) # 해상도 160*160 지정

      new_file_name = './Preprocess/FACE_VID/' + Clean_file_name + '.avi' 
      # new_file_name = re.sub(pattern='mp4', repl='avi', string=file_name)
      # save_file_name = './Preprocess/FACE_VID/%s.avi' %Clean_File_name
      out = cv2.VideoWriter(new_file_name, fourcc, fps, size)
      # out = cv2.VideoWriter(save_file_name, fourcc, fps, size)


      # 파일 처리 작업
      while (video2.isOpened()):
          ret, frame = video2.read()    

          if not ret:
              break            

          cropped = frame[y - int(h/3):y + h + int(h/3), 
                          x - int(w/3):x + w + int(w/3)]
          
          # cropped = frame[y:y+h, x:x+w]

          if cropped is not None:
            cropped = cv2.resize(cropped, 
                                dsize=size, 
                                interpolation=cv2.INTER_AREA)
            
            # 위에서 자른 얼굴 위치를 토대로 프레임 저장 (연속적 프레임이므로 영상 형태)
            out.write(cropped)
        
      # cropped 이미지 더 없으면 resize 함수 작동 중지 (혹시 모를 오류 방지)
      out.release()
      video2.release()
      
      return Clean_file_name    


In [ ]:
def sound_extraction(Clean_File):
  MOV_File = './Preprocess/CUT_VID/' + Clean_File + '.mp4'
  clip = VideoFileClip(MOV_File)

  sound_file_name = './Preprocess/WAV/' + Clean_File + '.wav'
  clip.audio.write_audiofile(sound_file_name)

In [ ]:
from moviepy.editor import *

In [ ]:
def sound_synthesis(Clean_File):
  WAV_File = './Preprocess/WAV/' + Clean_File + '.wav'
  print(WAV_File)
  
  MOV_File = './Preprocess/FACE_VID/' + Clean_File + '.avi'
  print(MOV_File)
  
  videoclip = VideoFileClip(MOV_File)
  audioclip = AudioFileClip(WAV_File)
  new_audioclip = CompositeAudioClip([audioclip])
  
  videoclip.audio = new_audioclip
  videoclip.write_videofile('./Preprocess/FACE_VID/' + Clean_File + '.mp4')

In [ ]:
import shutil

In [ ]:
def Preprocess(Clean_File_name):
  # 폴더 생성하기
    folder_num = Create_Folder_num(Clean_File_name)

    # 원본 영상, 원본 대본 불러오기
    vid_name = './main/' + Clean_File_name + '.mp4'
    script = './main/' + Clean_File_name + '.txt'

    # 유튜브 자동 자막에서 (start, end) 
    #시간 좌표 가져오기
    timeline = get_timeline(script)

    # 편집 완료한 대본 파일 -> './Preprocess/TXT' 폴더에 저장     
    get_script(script, Clean_File_name) 
    
    count = 0
    for t in timeline:
      start, end = t
  
      # 편집한 mp4 파일 -> 'Preprocess/CUT_VID' 폴더에 저장
      input_video_path = vid_name
      output_video_path = './Preprocess/CUT_VID/' + Clean_File_name + '_' + str(count) + '.mp4' 
  
      with VideoFileClip(input_video_path) as video:
          new = video.subclip(start, end)
          new.write_videofile(output_video_path)
      
      count += 1

    print('count :', count)

    for _ in range(0, count):
      # 인식 가능한 얼굴이 잘 나오는지 확인
      if int(Clean_File_name) <= 200:
        face_crop_success = face_crop_Pitchfork(Clean_File_name + '_' + str(_)) 
        if face_crop_success is not None:
            # mp4 파일에서 음성 추출 -> [Output] wav 파일 
            # './Preprocess/WAV' 폴더에 저장
            # sound_extraction(Clean_File_name + '_' + str(count))
            sound_extraction(face_crop_success)

            # 얼굴 부위만 crop -> [Output] 소리 없는 얼굴 crop 영상
            # './Preprocess/FACE_VID' 폴더에 저장
            # face_crop_vid(Clean_File_name + '_' + str(count))
                
            # 소리 없는 얼굴 crop 영상에 음성 합성
            # './Preprocess/FACE_VID' 폴더에 저장
            # sound_synthesis(Clean_File_name + '_' + str(count))
            sound_synthesis(face_crop_success)

            # 파일 생성
            folder_dir = './main/Pitchfork/' + str(folder_num)
            try: 
                if not os.path.exists(folder_dir):
                    os.makedirs(folder_dir)
                    print(folder_dir, '폴더 생성')
                    
            except OSError:
                print(folder_dir, ': 폴더 생성 오류')

            # fps 25로 영상 변환 -> 'main/생성일_변환숫자' 폴더에 저장     
            MOV_File = './Preprocess/FACE_VID/' + face_crop_success + '.mp4'
            clip = VideoFileClip(MOV_File)
            New_File = folder_dir + '/' + Clean_File_name + '_' + str(_) + '.mp4'
            clip.write_videofile(New_File, fps=25, codec="mpeg4")

            # 성공한 파일의 대본은 최종 폴더에 저장
            txt_file = './Preprocess/TXT/' + Clean_File_name + '_' +  str(_) + '.txt'
            if txt_file in glob.glob('./Preprocess/TXT/*.txt'):
              shutil.copy(txt_file, folder_dir)
              
              print('처리 완료')

        else:
          print('face_crop_vid 함수 적용 불가능 -> 다음 파일 처리 \n')
          continue


      else:
        face_crop_success = face_crop_Vogue(Clean_File_name + '_' + str(_))
        if face_crop_success is not None:
          # mp4 파일에서 음성 추출 -> [Output] wav 파일 
          # './Preprocess/WAV' 폴더에 저장
          # sound_extraction(Clean_File_name + '_' + str(count))
          sound_extraction(face_crop_success)

          # 얼굴 부위만 crop -> [Output] 소리 없는 얼굴 crop 영상
          # './Preprocess/FACE_VID' 폴더에 저장
          # face_crop_vid(Clean_File_name + '_' + str(count))
              
          # 소리 없는 얼굴 crop 영상에 음성 합성
          # './Preprocess/FACE_VID' 폴더에 저장
          # sound_synthesis(Clean_File_name + '_' + str(count))
          sound_synthesis(face_crop_success)

          # 파일 생성
          folder_dir = './main/Vogue/' + folder_num
          try: 
              if not os.path.exists(folder_dir):
                  os.makedirs(folder_dir)
                  print(folder_dir, '폴더 생성')
                  
          except OSError:
              print(folder_dir, ': 폴더 생성 오류')


          # fps 25로 영상 변환 -> 'main/생성일_변환숫자' 폴더에 저장     
          MOV_File = './Preprocess/FACE_VID/' + face_crop_success + '.mp4'
          clip = VideoFileClip(MOV_File)          
          New_File = folder_dir + '/' + Clean_File_name + '_' + str(_) + '.mp4'
          clip.write_videofile(New_File, fps=25, codec="mpeg4")

          # 성공한 파일의 대본은 최종 폴더에 저장
          txt_file = './Preprocess/TXT/' + Clean_File_name + '_' +  str(_) + '.txt'
          if txt_file in glob.glob('./Preprocess/TXT/*.txt'):
            shutil.copy(txt_file, folder_dir)
            
            print('처리 완료')
          


In [ ]:
# MOV_List : 영상 데이터 저장하는 변수 (전체 TXT 파일)
MOV_List = np.array(glob.glob('./main/*.mp4'))

# 전처리 범위 분할 : 3개 중에 하나 골라잡기 (0~100, 100~200, 200~)
# MOV_List = MOV_List[0:100]
# MOV_List = MOV_List[100:200] 
MOV_List = MOV_List[200:] 

print('MOV 파일 개수 :', len(MOV_List))
# print(MOV_List)

# TXT_List : 텍스트 데이터 저장하는 변수 (전체 MOV 파일)
TXT_List = np.array(glob.glob('./main/*.txt'))

# 전처리 범위 분할 : 3개 중에 하나 골라잡기 (0~100, 100~200, 200~)
# TXT_List = TXT_List[0:100]
# TXT_List = TXT_List[100:200]
TXT_List = TXT_List[200:]

print('TXT 파일 개수 :', len(TXT_List))


MOV 파일 개수 : 100
TXT 파일 개수 : 100


In [ ]:
Final_mov_list = np.array(glob.glob('./Preprocess/CUT_VID/*.mp4'))
Final_file_list = np.array([])

# 코랩의 경우 : '\\ -> /' 로 수정
# 주피터노트북의 경우 : '/ -> \\'로 수정
for file in Final_mov_list:
    file_name = re.sub(pattern='./Preprocess/CUT_VID/', repl='', string=file) 
    file_name = re.sub(pattern='.mp4', repl='', string=file_name)
    
    if "_" in file_name:
      file_name = re.sub(pattern='_\d+', repl='', string=file_name)
      # print(file_name)

      if file_name not in Final_file_list:    
        Final_file_list = np.append(Final_file_list, [file_name])

print(Final_file_list)
print('전처리 완료한 파일 개수 :', len(Final_file_list))

[]
전처리 완료한 파일 개수 : 0


In [ ]:
# 이 코드 cell의 목적 : 오류난 파일 skip하기 위한 용도
# 전처리 완료한 파일 이름만 남기는 작업
Clean_File_List = np.array([])

# [주의]
# 오류가 발생하는 경우에는
# need_MOV_List를 조절하여 다음 파일로 skip할 수 있습니다!
need_MOV_List = MOV_List[len(Final_file_list):]

# 코랩의 경우 : '\\ -> /' 로 수정
# 주피터노트북의 경우 : '/ -> \\'로 수정
for File in need_MOV_List:
    File_name = re.sub(pattern='./main/', repl='', string=File) #내 폴더는 \\로 나와서 수정함
    File_name = re.sub(pattern='.mp4', repl='', string=File_name)

    Clean_File_List = np.append(Clean_File_List, [File_name])
#     # if int(File_name) < 1000: #이부분을 수정?
#     if File_name not in Fianl_file_list:
#        Clean_File_List = np.append(Clean_File_List, [File_name])

print(Clean_File_List)
print('소속 위치, 확장자 뗀 본연의 파일명 개수 (전처리 X) : ', len(Clean_File_List))

['000039' '001008' '001090' '001205' '000048' '000083']
소속 위치, 확장자 뗀 본연의 파일명 개수 (전처리 X) :  6


In [ ]:
for Clean_File_name in notebook.tqdm(Clean_File_List):
    try:
      Preprocess(Clean_File_name)
    except Exception as e:
      print(e)


  0%|          | 0/6 [00:00<?, ?it/s]

000039 처리하는 중
[(16.47, 17.33), (17.33, 19.701), (19.701, 20.534), (20.534, 25.866), (25.866, 28.235), (28.235, 29.249), (37.496, 39.497), (39.497, 41.891), (41.891, 44.068), (44.068, 47.459), (47.459, 49.647), (49.647, 54.803)]
2
3
4
5
6
7
10
11
12
13
14
15
[MoviePy] >>>> Building video ./Preprocess/CUT_VID/000039_0.mp4
[MoviePy] Writing audio in 000039_0TEMP_MPY_wvf_snd.mp3



100%|██████████| 19/19 [00:00<00:00, 764.39it/s]

[MoviePy] Done.
[MoviePy] Writing video ./Preprocess/CUT_VID/000039_0.mp4




100%|██████████| 21/21 [00:00<00:00, 127.80it/s]


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: ./Preprocess/FACE_VID/000083_2.mp4 

[MoviePy] >>>> Building video ./main/Pitchfork/63633340800/000083_2.mp4
[MoviePy] Writing audio in 000083_2TEMP_MPY_wvf_snd.mp3



100%|██████████| 81/81 [00:00<00:00, 847.53it/s]

[MoviePy] Done.
[MoviePy] Writing video ./main/Pitchfork/63633340800/000083_2.mp4




100%|██████████| 92/92 [00:00<00:00, 1083.06it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: ./main/Pitchfork/63633340800/000083_2.mp4 

처리 완료
